In [1]:
import time
import math
import numpy as np
from numpy.fft  import fft2, ifft2
from scipy.signal import convolve2d

# Data Generation

In [2]:
rows = 7
cols = 7
mu, sigma = 0, math.sqrt(2.0 / (rows * cols))

k_size = 3

filter2d = np.random.normal(mu, sigma, size=(k_size, k_size))

image = np.random.randint(255, size=(rows, cols))

print("filter = ", filter2d)
print("image = ", image)

filter =  [[ 0.06734867  0.04643565 -0.11393147]
 [ 0.2554598  -0.42890386 -0.00367388]
 [ 0.01225916 -0.09873612 -0.10296112]]
image =  [[102  23 103 242  38  89  55]
 [176 208  19  34 241 176  95]
 [121 238 118  76 118  71 138]
 [ 47  57 121  66 201  45 236]
 [111  85  97   0 137  53  62]
 [172 210 131 217 140  38 240]
 [116  37 105  17 197  71 226]]


## Scipy Convolution

Automatically chooses direct or Fourier method based on an estimate
of which is faster (default)

In [3]:
start = time.time()
scipy_res = convolve2d(image, filter2d, mode='same', boundary='fill', fillvalue=0)
end = time.time()
print("Finish in %.2f ms" % ((end - start)*1000))
print("scipy_res = ", scipy_res)

Finish in 1.28 ms
scipy_res =  [[-1.56914206e+01  6.95913528e+00 -2.96594021e+00 -7.88205274e+01
   2.47191848e+01 -3.71484491e+01 -3.95572400e+01]
 [-1.04928682e+01 -9.13020216e+01 -2.63167015e+01  1.09123264e+01
  -8.45049461e+01 -6.49824527e+01 -5.76675104e+01]
 [ 9.57613544e-02 -1.05363506e+02 -5.13755491e+01 -2.42865937e+00
  -5.30459182e+01 -4.15751413e+01 -8.11187699e+01]
 [-3.74767030e+00 -3.03870107e+01 -7.56502614e+01  2.56386418e+00
  -8.36308051e+01  1.38095718e+01 -1.25481818e+02]
 [-7.70591894e+00 -2.20908527e+01 -6.21505815e+01  2.27111646e+01
  -8.69730815e+01 -2.76640371e+01 -4.79065705e+01]
 [-2.21641522e+01 -8.02949217e+01 -1.80470368e+01 -6.40024781e+01
  -5.20206696e+01  2.19922732e+01 -1.12249791e+02]
 [-5.47090256e+01 -2.63102820e+01 -7.27240519e+01  9.45109456e+00
  -1.02118643e+02  1.13336551e+01 -1.24802310e+02]]


## My implementation

In [4]:
def conv2d(x, f):
    r, c = x.shape
    kr, kc = f.shape
    res = np.zeros(x.shape)
    for i in range(r):
        for j in range(c):
            # (i, j) is the center position of filter
            for ki in range(-int(kr / 2), int(kr / 2) + 1, 1):
                for kj in range(-int(kc / 2), int(kc / 2) + 1, 1):
                    m = i - ki
                    n = j - kj
                    #print(ki, kj, i, j)
                    if m >= 0 and m < r and n >= 0 and n < c:
                        res[i, j] += x[m, n] * f[ki + int(kr / 2), kj + int(kc / 2)]
    return res
start = time.time()
res = conv2d(image, filter2d)
end = time.time()
print("Finish in %.2f ms" % ((end - start)*1000))
print("my_res = ", res)

Finish in 3.68 ms
my_res =  [[-1.56914206e+01  6.95913528e+00 -2.96594021e+00 -7.88205274e+01
   2.47191848e+01 -3.71484491e+01 -3.95572400e+01]
 [-1.04928682e+01 -9.13020216e+01 -2.63167015e+01  1.09123264e+01
  -8.45049461e+01 -6.49824527e+01 -5.76675104e+01]
 [ 9.57613544e-02 -1.05363506e+02 -5.13755491e+01 -2.42865937e+00
  -5.30459182e+01 -4.15751413e+01 -8.11187699e+01]
 [-3.74767030e+00 -3.03870107e+01 -7.56502614e+01  2.56386418e+00
  -8.36308051e+01  1.38095718e+01 -1.25481818e+02]
 [-7.70591894e+00 -2.20908527e+01 -6.21505815e+01  2.27111646e+01
  -8.69730815e+01 -2.76640371e+01 -4.79065705e+01]
 [-2.21641522e+01 -8.02949217e+01 -1.80470368e+01 -6.40024781e+01
  -5.20206696e+01  2.19922732e+01 -1.12249791e+02]
 [-5.47090256e+01 -2.63102820e+01 -7.27240519e+01  9.45109456e+00
  -1.02118643e+02  1.13336551e+01 -1.24802310e+02]]


## FFT convolution

In [5]:
from numpy.fft  import fft2, ifft2
def np_fftconvolve(A, B):
    return np.real(ifft2(fft2(A)*fft2(B, s=A.shape)))
start = time.time()
fft_res = np_fftconvolve(image, filter2d)
end = time.time()
print("Finish in %.2f ms" % ((end - start)*1000))
print("fft_res = ", res)

Finish in 2.23 ms
fft_res =  [[-1.56914206e+01  6.95913528e+00 -2.96594021e+00 -7.88205274e+01
   2.47191848e+01 -3.71484491e+01 -3.95572400e+01]
 [-1.04928682e+01 -9.13020216e+01 -2.63167015e+01  1.09123264e+01
  -8.45049461e+01 -6.49824527e+01 -5.76675104e+01]
 [ 9.57613544e-02 -1.05363506e+02 -5.13755491e+01 -2.42865937e+00
  -5.30459182e+01 -4.15751413e+01 -8.11187699e+01]
 [-3.74767030e+00 -3.03870107e+01 -7.56502614e+01  2.56386418e+00
  -8.36308051e+01  1.38095718e+01 -1.25481818e+02]
 [-7.70591894e+00 -2.20908527e+01 -6.21505815e+01  2.27111646e+01
  -8.69730815e+01 -2.76640371e+01 -4.79065705e+01]
 [-2.21641522e+01 -8.02949217e+01 -1.80470368e+01 -6.40024781e+01
  -5.20206696e+01  2.19922732e+01 -1.12249791e+02]
 [-5.47090256e+01 -2.63102820e+01 -7.27240519e+01  9.45109456e+00
  -1.02118643e+02  1.13336551e+01 -1.24802310e+02]]
